In [1]:
import nltk
#nltk.download()
from __future__ import unicode_literals
from hazm import *
import re

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#importing data from file
df=pd.read_excel('failure_description.xlsx')

In [4]:
#looking to dataframe
df.iloc[:,1][0]

'بازو بیرونی شماره21و6 گریسکاری نمیشود'

In [5]:
#func for removing english character
def remove_eng(x):
    return re.sub('[0-9a-zA-Z]',' ',x)

In [6]:
#mapping function to dataframe
df['Problem_Description'] = df.iloc[:,1].map(lambda x: remove_eng(x))

In [7]:
#looking
df

,Cause_of_failure,Problem_Description
0,NaN,بازو بیرونی شماره و گریسکاری نمیشود
1,NaN,نازل اب روی تجهیز گرفته است
2,NaN,تجهیز الارم دارد
3,NaN,دستگاه گلس بید یکی از عملکرد ان در هنگام کار ...
4,NaN,دستگاه گلس بید استارت نمیشود.
...,...,...
59387,NaN,بالا رفتن دمای یاتاقان پنیون بال میل.
59388,NaN,نیاز به نصب داربست در اطراف بدنه بال میل.
59389,سایش,شل بودن وی بلت
59390,NaN,درخواست ساخت و تعویض داکت ماقبل و بعد از نازل...


In [8]:
#useing HAZM for normalizing sentenses
normalizer = Normalizer(token_based=True)
def norm(x):
    return normalizer.normalize(x)

In [9]:
#sample test 
norm(df.iloc[:,1][0])

'بازو بیرونی شماره و گریسکاری نمیشود'

In [10]:
#mapping function to dataframe
df['Normalized_description'] = df.iloc[:,1].map(lambda x: norm(x))

In [11]:
#useing HAZM for tokenizing
def word_tok(x):
    return word_tokenize(x)
df['word_tokenize'] = df.iloc[:,2].map(lambda x: word_tok(x))

In [12]:
#looking
df.iloc[:,3][0]

['بازو', 'بیرونی', 'شماره', 'و', 'گریسکاری', 'نمیشود']

In [13]:
#removing empty item and space from begin and end of words
def remove_space(x):
    remove_space = [item.strip() for item in x]
    delete_empty = [item for item in remove_space if item.strip()]
    return delete_empty
    
df['removed_space'] = df.iloc[:,3].map(lambda x: remove_space(x))        

In [14]:
#save cleaned dataframe to excel file
#df.to_excel('temp.xlsx')

In [15]:
#sample test
a=[' یتبنیمشس سینش  ','','نسشیت    ']
print(a)
remove_space(a)

[' یتبنیمشس سینش  ', '', 'نسشیت    ']


['یتبنیمشس سینش', 'نسشیت']

In [16]:
#removing stopwords
stop_words_df=pd.read_excel('stop_words.xlsx')
stop_words = stop_words_df.iloc[:,0].to_list()
def remove_stopwords(x):
    output=[]
    for item in x:
        if item not in stop_words:
            output.append(item)
    return output

In [17]:
#mapping function to dataframe
df['removed_stopwords'] = df.iloc[:,4].map(lambda x: remove_stopwords(x))

In [18]:
#making sentence by joining
df['sentens'] = df.iloc[:,5].map(lambda x: ' '.join(x))

In [19]:
#making new dataframe by selecting some colume from old dataframe
temp=df[['Cause_of_failure','sentens','removed_stopwords']].copy()

In [20]:
#removing NaN
final_df=temp[temp['Cause_of_failure'].notnull()]

In [21]:
#looking
final_df

,Cause_of_failure,sentens,removed_stopwords
5,شکستگی,شفت برگشتی ستون شکسته_است,"[شفت, برگشتی, ستون, شکسته_است]"
13,عدم تنظیم,تعمیر جنوب,"[تعمیر, جنوب]"
19,انحراف,رفع انحراف نوار,"[رفع, انحراف, نوار]"
24,عیب هیدرولیکی,الارم تجهیز دلیل کاهش فشار پمپ گریس اتفاق افتا...,"[الارم, تجهیز, دلیل, کاهش, فشار, پمپ, گریس, ات..."
33,شکستگی,پایپ جنوب چکش ثابت افتاده_است,"[پایپ, جنوب, چکش, ثابت, افتاده_است]"
...,...,...,...
59360,سایش,گپ رول شمارش پایین تنظیم خارج‌شده گشاد تنظیم م...,"[گپ, رول, شمارش, پایین, تنظیم, خارج‌شده, گشاد,..."
59361,سایش,گپ رول شمارش پایین تنظیم خارج‌شده گشاد تنظیم م...,"[گپ, رول, شمارش, پایین, تنظیم, خارج‌شده, گشاد,..."
59362,سایش,گپ رول شمارش پایین تنظیم خارج‌شده گشاد تنظیم م...,"[گپ, رول, شمارش, پایین, تنظیم, خارج‌شده, گشاد,..."
59363,سایش,گپ رول شمارش پایین تنظیم خارج‌شده گشاد تنظیم م...,"[گپ, رول, شمارش, پایین, تنظیم, خارج‌شده, گشاد,..."


In [22]:
#changing categure value by 0,1,2,....
res = dict(zip(final_df.Cause_of_failure.value_counts().keys().to_list(),range(27)))
final_df.Cause_of_failure.replace(res,inplace = True)

C:\Users\h.alavi\AppData\Local\Temp\ipykernel_10300\3887569738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.Cause_of_failure.replace(res,inplace = True)


In [23]:
#changed by this values
res.items()

dict_items([('سایش', 0), ('عدم عملکرد صحیح', 1), ('شل بودن', 2), ('عدم تنظیم', 3), ('شکستگی', 4), ('انحراف', 5), ('افزایش ویبره', 6), ('پایان عمر مفید', 7), ('صدای غیر عادی', 8), ('پارگی', 9), ('دفرمگی', 10), ('نشتی روغن', 11), ('الودگی زیاد', 12), ('گرفتگی', 13), ('سوختگی', 14), ('نشتی اب', 15), ('ترک خوردگی', 16), ('افزایش دما', 17), ('نشتی هوا', 18), ('زنگ زدگی', 19), ('خرابی نسوز و عایق', 20), ('فشار غیرعادی', 21), ('نشتی گاز', 22), ('کمبود روغن', 23), ('دفورمگی', 24), ('عیب هیدرولیکی', 25), ('ترکیدگی', 26)])

In [24]:
#reindexing dataframe
final_df.reset_index(inplace= True,drop=True)

In [25]:
#looking
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6594 entries, 0 to 6593
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Cause_of_failure   6594 non-null   int64 
 1   sentens            6594 non-null   object
 2   removed_stopwords  6594 non-null   object
dtypes: int64(1), object(2)
memory usage: 154.7+ KB


In [26]:
#save cleaned dataframe to excel file
#final_df.to_excel('f_data.xlsx')

In [27]:
#Vectorizing data by sklearn "Bag Of Words"
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [28]:
#appling model to dataframe
bow = cv.fit_transform(final_df.iloc[:,1]).toarray()

In [29]:
bow.shape

(6594, 2585)

In [30]:
#Vectorizing data by sklearn "TF-idf"
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
#making class
tf = TfidfVectorizer()

In [32]:
#training data
tf.fit(final_df.iloc[:,1])

TfidfVectorizer()

In [33]:
#looking to vocabulary
print(tf.fit(final_df.iloc[:,1]).vocabulary_)

{'شفت': 1310, 'برگشتی': 429, 'ستون': 1163, 'شکسته_است': 1341, 'تعمیر': 590, 'جنوب': 697, 'رفع': 1000, 'انحراف': 241, 'نوار': 1911, 'الارم': 196, 'تجهیز': 531, 'دلیل': 899, 'کاهش': 2386, 'فشار': 1466, 'پمپ': 2274, 'گریس': 2519, 'اتفاق': 23, 'افتاده_است': 183, 'پایپ': 2234, 'چکش': 2348, 'ثابت': 659, 'نصب': 1877, 'اسکافلد': 135, 'جهت': 702, 'پرژکتور': 2258, 'جلو': 688, 'پولپیت': 2296, 'بالینگ': 372, 'هماهنگ': 1970, 'اقای': 191, 'ثقفی': 660, 'گیربکس': 2554, 'روغن': 1023, 'پیچ': 2316, 'النی': 206, 'بازوی': 347, 'بالایی': 369, 'بیرون_امده': 503, 'نشتی': 1870, 'اب': 0, 'سقف': 1194, 'لوله': 1587, 'فاضلاب': 1446, 'پست': 2260, 'لطفا': 1569, 'بررسی': 406, 'عمرانی': 1421, 'انجام': 239, 'شل_شدگی': 1314, 'براکت': 395, 'ساپورت': 1141, 'ویل': 2203, 'بیرونی': 504, 'غرب': 1439, 'مشاهده': 1691, 'گردید': 2503, 'دبل_فلاپ_دمپر': 820, 'شمال': 1320, 'درام': 834, 'استراکچر': 98, 'همراه': 1975, 'دیگه': 954, 'تجهیزات': 532, 'میل': 1807, 'بلت': 461, 'خرابی': 758, 'ریترن': 1044, 'ایدلر': 302, 'اسکریت_برد': 150, 'ج

In [34]:
#making vectors by TF-idf
X=tf.transform(final_df.iloc[:,1]).toarray()

In [35]:
y=final_df['Cause_of_failure']

In [36]:
X[0].sum()

1.9637358669133107

In [37]:
#spliting data to train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)

In [38]:
X_train.shape

(5275, 2585)

In [39]:
#training model 
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB().fit(X_train,y_train)

In [40]:
#predicting 
y_pred=model.predict(X_test)

In [41]:
#calculating accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.5807429871114481

In [42]:
#calculating accuracy
y_predict=model.predict(X_train)
accuracy_score(y_predict,y_train)

0.6037914691943128

In [43]:
#Indexing words of corpus by tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [44]:
#Indexing
max_words = 500
tkn = Tokenizer(num_words=max_words)
tkn.fit_on_texts(final_df.iloc[:,1])
seq = tkn.texts_to_sequences(final_df.iloc[:,1])
print(len(seq))
for i in range(10):
    print(seq[i],'<----->',final_df.iloc[:,1][i])

6594
[32, 413, 287, 71, 15] <-----> شفت برگشتی ستون شکسته_است
[127, 2] <-----> تعمیر جنوب
[40, 19, 47] <-----> رفع انحراف نوار
[376, 34, 60, 377, 282, 388, 455, 38, 15] <-----> الارم تجهیز دلیل کاهش فشار پمپ گریس اتفاق افتاده_است
[295, 2, 222, 223, 38, 15] <-----> پایپ جنوب چکش ثابت افتاده_است
[40, 19, 47] <-----> رفع انحراف نوار
[13, 81, 9, 127, 362, 151, 315, 339] <-----> نصب اسکافلد جهت تعمیر پرژکتور جلو پولپیت بالینگ هماهنگ اقای ثقفی
[39, 90] <-----> گیربکس روغن
[25, 155, 38, 15] <-----> پیچ النی افتاده_است
[58, 78, 141, 101] <-----> بازوی بالایی بیرون_امده


In [46]:
#calculating Maximum Len
max_len=0
for item in seq:
    if len(item)>max_len:
        max_len=len(item)
print(max_len)

42


In [47]:
#making list with equal len
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_docs = pad_sequences(seq, padding = 'pre', maxlen= max_len)
print(padded_docs)

[[  0   0   0 ... 287  71  15]
 [  0   0   0 ...   0 127   2]
 [  0   0   0 ...  40  19  47]
 ...
 [  0   0   0 ... 454  18  83]
 [  0   0   0 ... 454  18  83]
 [  0   0   0 ...   0  30  26]]


In [52]:
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
embedding_vector_features = 10
model = Sequential()
model.add(Embedding(max_words, embedding_vector_features, input_length=max_len))
model.add(LSTM(256))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 42, 10)            5000      
                                                                 
 lstm (LSTM)                 (None, 256)               273408    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 278,665
Trainable params: 278,665
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
model.fit(padded_docs,y,batch_size=128,epochs=10, validation_split=0.2)

Epoch 1/10
42/42 [==============================] - 6s 106ms/step - loss: -35.4901 - accuracy: 0.1767 - val_loss: -53.3398 - val_accuracy: 0.1865
Epoch 2/10
42/42 [==============================] - 4s 97ms/step - loss: -71.4597 - accuracy: 0.1723 - val_loss: -75.2133 - val_accuracy: 0.1865
Epoch 3/10
42/42 [==============================] - 4s 97ms/step - loss: -95.3519 - accuracy: 0.1723 - val_loss: -95.9748 - val_accuracy: 0.1865
Epoch 4/10
42/42 [==============================] - 4s 97ms/step - loss: -118.1550 - accuracy: 0.1723 - val_loss: -115.9633 - val_accuracy: 0.1865
Epoch 5/10
42/42 [==============================] - 4s 98ms/step - loss: -140.5243 - accuracy: 0.1723 - val_loss: -136.1362 - val_accuracy: 0.1865
Epoch 6/10
42/42 [==============================] - 4s 92ms/step - loss: -163.0596 - accuracy: 0.1723 - val_loss: -156.4582 - val_accuracy: 0.1865
Epoch 7/10
42/42 [==============================] - 4s 91ms/step - loss: -185.7374 - accuracy: 0.1723 - val_loss: -176.5411

In [ ]:
#prediction 
text='جک نشتی دارد'.strip().split()
print(text)
pr=pad_sequences(tkn.texts_to_sequences([text]), padding = 'pre', maxlen= max_len)
print(pr)
model.predict(pr)

In [ ]:
#bag of words useing sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer=CountVectorizer()
vocabulary=vectorizer.fit(final_df.iloc[:,1])

In [ ]:
X= vectorizer.transform(final_df.iloc[:,1])

In [ ]:
X.toarray()

In [ ]:
print(vocabulary.get_feature_names())

In [ ]:
for item in final_df.iloc[:,1]:
    if item.find(' یکسر') != -1:
        print(item)
        print(final_df[final_df['sentens']== item])

In [ ]:
final_df

In [ ]:
from googletrans import Translator

In [ ]:
tr = Translator()

In [ ]:
tr.translate('contents', dest='en')